### Header

In [ ]:
# import libraries

# maths
import numpy as np
import pandas as pd
#import scipy.stats as stats
#from pandas.api.types import is_numeric_dtype

# visual
#from matplotlib_venn import venn2
#import seaborn as sns
#import matplotlib.pyplot as plt
#%matplotlib inline

# modelling
# from sklearn import linear_model
# from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression
# from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
# from sklearn.preprocessing import StandardScaler,PolynomialFeatures
# from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
# from sklearn.pipeline import make_pipeline,Pipeline

# nlp
# from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from nltk.stem import PorterStemmer,WordNetLemmatizer
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# import spacy
# from spacy.tokens import Doc

# web
#import requests
#import json

# others
import os
import re
import time
import datetime as datetime

In [ ]:
# file paths

input_path = '../data/2_input/'
clean_path = '../data/3_clean/'
output_path = '../data/4_output/'

image_path = '../images/'

### Import Data

In [ ]:
train = pd.read_csv(input_path+'train.csv')
test = pd.read_csv(input_path+'test.csv')

### Inspect Data

In [ ]:
train.head()

In [ ]:
test.head()

### Clean Data

In [ ]:
def create_yr(x): 
    return x.split('-')[0] 

def create_mth(x): 
    return x.split('-')[1] 

def create_day(x): 
    return x.split('-')[2] 

def rename_columns (columns):
    return [column.lower() for column in columns]

def clean_data(df): 
    df['year'] = df.Date.apply(create_yr)
    df['month'] = df.Date.apply(create_mth)
    df['day'] = df.Date.apply(create_day)
    
    #df['latitude'] = df.Latitude.apply(int)
    #df['longitude'] = df.Longitude.apply(int)
    df['latitude'] = df.Latitude
    df['longitude'] = df.Longitude
    
    df.drop(['Address', 'AddressNumberAndStreet', 'AddressAccuracy', 'Date', 'Latitude', 'Longitude'], axis = 1, inplace = True)
    df.columns = rename_columns(df.columns)
    return df

train = clean_data(train)
test = clean_data(test)

In [ ]:
#train[train.nummosquitos==50].shape

#train[train.nummosquitos==50]

#train[train.duplicated()]

#col = [col for col in train.columns if col not in ['nummosquitos','wnvpresent']]

#col

#train.groupby(train.columns.tolist(),as_index=False).size()

#train.duplicated(col).sum()

#train[train.duplicated(col)]

#test.duplicated().sum()

In [ ]:
# cols = ['species','trap','year','month', 'day','latitude','longitude']
# cols = ['species','trap','year','month', 'day']

# mask_dulipcated = train.duplicated(subset=cols, keep=False)

# print(train[mask_dulipcated].shape)
# train[mask_dulipcated]

In [ ]:
# merge nummosquitos and wnvpresent for duplicated rows

train_2 = train.copy()

cols = ['species','trap','year','month', 'day','latitude','longitude']

for row_idx,row in train_2.iterrows():
    
    if row_idx > 0:
        
        duplicate_count = 0        
        
        for col in cols:
            
            if train_2.at[row_idx,col] == train_2.at[row_idx-1,col]:
                duplicate_count += 1               
                
        if duplicate_count == len(cols):
            train_2.at[row_idx,'nummosquitos'] = train_2.at[row_idx-1,'nummosquitos'] + train_2.at[row_idx,'nummosquitos']
            train_2.at[row_idx,'wnvpresent'] = train_2.at[row_idx-1,'wnvpresent'] + train_2.at[row_idx,'wnvpresent']

# remove duplicated rows (keep only last row)
train_2.drop_duplicates(subset=cols, keep='last',inplace=True)

In [ ]:
# check total nummosquitos and wnvpresent

print(train['nummosquitos'].sum())
print(train['wnvpresent'].sum())
print('')
print(train_2['nummosquitos'].sum())
print(train_2['wnvpresent'].sum())
print('')
print(train_2['wnvpresent'].unique())

In [ ]:
# convert wnvpresent: total count to 0 or 1

for row_idx,row in train_2.iterrows():
    
    if row['wnvpresent'] > 1:
        #print(row_idx,row['wnvpresent'])
        train_2.at[row_idx,'wnvpresent'] = 1
        
print(train_2['wnvpresent'].sum())
print(train_2['wnvpresent'].unique())

### Output Data

In [ ]:
train_2.head()

In [ ]:
test.head()

In [ ]:
train_2.to_csv(clean_path+'train_clean.csv',index=False)
test.to_csv(clean_path+'test_clean.csv',index=False)